In [1]:
%cd ..

/Users/juansegundohevia/Documents/Research/Summer24 RA/IDIL


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import dotenv
import os
dotenv.load_dotenv(dotenv.find_dotenv())

True

In [5]:
from idil_algs.IDIL.train import load_expert_data_w_labels

In [16]:
data_path = "idil_train/experts/CleanupSingle-v0_100.pkl"


expert_dataset, traj_labels, cnt_label = load_expert_data_w_labels(data_path, 3, 2, 42)

--> Expert memory size: 134
num_labeled: 2 / 3, num_samples:  134


In [17]:
expert_dataset.trajectories.keys()

dict_keys(['states', 'next_states', 'actions', 'latents', 'rewards', 'dones', 'lengths'])

In [20]:
expert_dataset.trajectories["actions"][0][:5] # sample actions

[0, 5, 0, 4, 3]

In [21]:
expert_dataset.trajectories["states"][0][:5] # sample actions

[27, 0, 9, 9, 9]

In [22]:
expert_dataset.trajectories["latents"][0][:5] # sample actions

[0, 0, 3, 3, 3]

In [33]:
from collections import defaultdict
# print shapes of upper level data
for idx in range(3):
    print(f"Trajectory {idx}:")
    print("Num. States: ", len(expert_dataset.trajectories["states"][idx]))
    print("Num. Actions: ", len(expert_dataset.trajectories["actions"][idx]))
    print("Num. Latents: ", len(expert_dataset.trajectories["latents"][idx]))


def create_expert_action_policy(dataset):
    """
    Parse state, actions and latent lists and zip into a dictionary where each key is
    a tuple of (state, latent) and the values is a list of all actions taken in that state-latent pair.
    """
    states = dataset.trajectories["states"]
    actions = dataset.trajectories["actions"]
    latents = dataset.trajectories["latents"]

    state_latent_actions = defaultdict(list)
    for traj_idx in range(len(states)):
        assert len(states[traj_idx]) == len(actions[traj_idx]) == len(latents[traj_idx])
        _states_idx = states[traj_idx]
        _actions_idx = actions[traj_idx]
        _latents_idx = latents[traj_idx]

        for state, action, latent in zip(_states_idx, _actions_idx, _latents_idx):
            state_latent_actions[(state, latent)].append(action)

    return state_latent_actions
        

exp_policy = create_expert_action_policy(expert_dataset)
exp_policy[(0,0)]

Trajectory 0:
Num. States:  42
Num. Actions:  42
Num. Latents:  42
Trajectory 1:
Num. States:  41
Num. Actions:  41
Num. Latents:  41
Trajectory 2:
Num. States:  51
Num. Actions:  51
Num. Latents:  51


[5, 5, 2, 5]

In [23]:
class ExpertPolicySampler:
    """
    Expert policy sampler used for discrete action spaces only.
    This creates a dictionary where keys are (state, latent) pairs and provides a 
    choose_action method to sample uniformly from seen actions in that state-latent pair.
    """
    def __init__(self, data_path, num_trajectories, num_labeled, seed=42):
        self.data_path = data_path
        self.num_trajectories = num_trajectories
        self.num_labeled = num_labeled
        self.seed = seed

        self.expert_dataset, self.traj_labels, self.cnt_label = load_expert_data_w_labels(data_path, num_trajectories, num_labeled, seed)

        self.expert_policy = self._create_expert_policy()

    def _create_expert_policy(self):
        """
        Parse state, actions and latent lists from dataset and create a dictionary where 
        keys are (state, latent) pairs and values are lists of actions taken in that state-latent pair.
        """
        states = self.expert_dataset.trajectories["states"]
        actions = self.expert_dataset.trajectories["actions"]
        latents = self.expert_dataset.trajectories["latents"]

        state_latent_actions = defaultdict(list)
        for traj_idx in range(len(states)):
            assert len(states[traj_idx]) == len(actions[traj_idx]) == len(latents[traj_idx])
            _states_idx = states[traj_idx]
            _actions_idx = actions[traj_idx]
            _latents_idx = latents[traj_idx]

            for state, action, latent in zip(_states_idx, _actions_idx, _latents_idx):
                state_latent_actions[(state, latent)].append(action)

        return state_latent_actions
    
    def choose_action(self, state, latent):
        """
        Sample an action from the expert policy given a state and latent.
        """
        return np.random.choice(self.expert_policy[(state, latent)])
    

KeyError: 'prev_latents'